In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from .maddpg_agent import MADDPG_Net
from .environment import Environment

## Set hyper parameters

In [ ]:
class args:
    eval = False
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_episodes = 1200
    pretrain_length = 25000
    batch_size = 128
    buffer_size = int(3e5)
    actor_learn_rate = 5e-4
    critic_learn_rate = 3e-4
    gamma = 0.99
    tau = 1e-4
    e = 0.3
    e_decay = 1
    e_min = 0.00
    anneal_max = 0.7
    rollout = 5
    num_atoms = 100
    vmin = 0.0
    vmax = 2.0
    update_every = 2500
    print_every = 100
    update_type = 'soft'
    random_seed = 0
    train = not eval

## Set Environment

In [ ]:
env_filepath = "./Tennis_Windows_x86_64/Tennis.exe"
env = Environment(env_filepath)

multi_agent = MADDPG_Net(env, args)

## Train the agents

In [ ]:
def train_maddpg(multi_agent, args, env):
    multi_agent.initialize_memory(args.pretrain_length, env)
    scores_deque = deque(maxlen=args.print_every)
    scores = []
    highest_avg_score = 0

    for episode in range(1, args.num_episodes + 1):
        env.reset()
        obs = env.states
        score = np.zeros(multi_agent.agent_count)
        while True:
            actions = multi_agent.act(obs)
            next_obs, rewards, dones = env.step(actions)
            score += rewards
            multi_agent.store((obs, next_obs, actions, rewards, dones))
            multi_agent.learn()
            obs = next_obs
            if np.any(dones):
                break
        scores_deque.append(np.max(score))
        scores.append(np.max(score))
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_deque)), end="")
        if np.mean(scores_deque) >= highest_avg_score:
            for i, a in enumerate(multi_agent.agents):
                torch.save(a.actor_local.state_dict(), f'checkpoint_actor_a{i}.pth')
                torch.save(a.critic_local.state_dict(), f'checkpoint_critic_a{i}.pth')
            highest_avg_score = np.mean(scores_deque)
        if episode % args.print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_deque)))
        multi_agent.new_episode(scores)
    env.close()
    return scores

In [ ]:
scores = train_maddpg(multi_agent, args, env)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores) + 1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

## Load Network
Load saved network for evaluation agents.

In [ ]:
def load_qnetwork(multi_agent, env):
    for i, a in enumerate(multi_agent.agents):
        a.actor_local.load_state_dict(torch.load(f'checkpoint_actor_a{i}.pth'))
        a.critic_local.load_state_dict(torch.load(f'checkpoint_critic_a{i}.pth'))

    scores = []
    for episode in range(1, 1):
        env.reset(train=False)
        obs = env.states
        score = np.zeros(multi_agent.agent_count)
        while True:
            actions = multi_agent.act(obs, training=False)
            next_obs, rewards, dones = env.step(actions)
            obs = next_obs
            score += rewards
            if np.any(dones):
                break
        scores.append(np.max(score))
        print("Score: {}".format(np.max(score)))
        multi_agent.new_episode(scores)
    env.close()
    return

In [ ]:
load_qnetwork(multi_agent, env)